In [1]:
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import time

from PIL import Image as im

In [2]:
file_path = "/home/tigeriv/Data/Carla/Cartesian/Test/Town10_Medium/cartesian/ConvBKI/"
# file_path = "/home/tigeriv/Data/KITTI/dataset/sequences/08/ConvBKIKITTI/"
# file_path = "/home/tigeriv/Data/KITTI/dataset/sequences/08/ConvBKIKITTI_Transfer/"
# eval_size = (256, 256, 64)

KITTI_COLOR = True

In [3]:
COLOR_MAP = {
    0: np.asarray([255, 255, 255]), # Free
    1: np.asarray([255, 255, 255]), # Building
    2: np.asarray([255, 255, 255]), # Barrier
    3: np.asarray([255, 255, 255]), # Other
    4: np.asarray([255, 255, 255]), # Pedestrian
    5: np.asarray([255, 255, 255]), # Pole
    6: np.asarray([255, 255, 255]), # Road
    7: np.asarray([255, 255, 255]), # Ground
    8: np.asarray([255, 255, 255]), # Sidewalk
    9: np.asarray([255, 255, 255]), # Vegetation
    10: np.asarray([255, 255, 255]) # Vehicle
}

LABEL_COLORS = np.array([
    (255, 255, 255), # None
    (70, 70, 70),    # Building
    (100, 40, 40),   # Fences
    (55, 90, 80),    # Other
    (255, 255, 0),   # Pedestrian
    (153, 153, 153), # Pole
    (0, 0, 255),  # Road
    (100, 100, 100),  # Ground
    (150, 10, 10),  # Sidewalk
    (0, 155, 0),  # Vegetation
    (255, 0, 0)     # Vehicle
]).astype(np.uint8)

In [4]:
KITTI_COLORS = np.array([
    (  255,   255,   255,),
    (245, 150, 100,),
    (245, 230, 100,),
    (150,  60,  30,),
    (180,  30,  80,),
    (250,  80, 100,),
    ( 30,  30, 255,),
    (200,  40, 255,),
    ( 90,  30, 150,),
    (255,   0, 255,),
    (255, 150, 255,),
    ( 75,   0,  75,),
    ( 75,   0, 175,),
    (  0, 200, 255,),
    ( 50, 120, 255,),
    (  0, 175,   0,),
    (  0,  60, 135,),
    ( 80, 240, 150,),
    (150, 240, 255,),
    (  0,   0, 255,),
]).astype(np.uint8)

kitti_to_carla = {
    0: 0,
    1: 10,
    2: 10,
    3: 10,
    4: 10,
    5: 10,
    6: 4,
    7: 4,
    8: 4,
    9: 6,
    10: 6,
    11: 8,
    12: 7,
    13: 1,
    14: 9,
    15: 9,
    16: 7,
    17: 7,
    18: 5,
    19: 5
}

if KITTI_COLOR:
    for kitti_i in kitti_to_carla.keys():
        carla_i = kitti_to_carla[kitti_i]
        kitti_color_i = KITTI_COLORS[kitti_i]
        LABEL_COLORS[carla_i, :] = KITTI_COLORS[kitti_i, :]
    LABEL_COLORS[6, :] = KITTI_COLORS[9, :]
    LABEL_COLORS[7, :] = KITTI_COLORS[17, :]
    LABEL_COLORS[10, :] = KITTI_COLORS[1, :]
    LABEL_COLORS[0, :] = np.asarray([255, 255, 255])
print(LABEL_COLORS/255)

[[1.         1.         1.        ]
 [0.         0.78431373 1.        ]
 [0.39215686 0.15686275 0.15686275]
 [0.21568627 0.35294118 0.31372549]
 [0.35294118 0.11764706 0.58823529]
 [0.         0.         1.        ]
 [1.         0.         1.        ]
 [0.31372549 0.94117647 0.58823529]
 [0.29411765 0.         0.29411765]
 [0.         0.68627451 0.        ]
 [0.96078431 0.58823529 0.39215686]]


In [11]:
# Form BEV
def form_bev(labels, counts):
    bev_map = np.zeros((eval_size[0], eval_size[1]))
    for x in range(eval_size[0]):
        for y in range(eval_size[1]):
            label_xy = None
            for z in range(eval_size[2]-1, -1, -1):
                count_z = counts[x, y, z]
                if count_z != 0 and labels[x, y, z] != 0:
                    intersected = True
                    label_xy = labels[x, y, z]
                    break
            if label_xy:
                remapped_xy = LABELS_REMAP[label_xy]
                bev_map[x, y] = remapped_xy
    return bev_map

def plot_bev(bev_map):
    plt.figure()
    ax = sns.heatmap(bev_map, cmap="YlGnBu", xticklabels=False, yticklabels=False)
    ax.invert_yaxis()
    ax.set_xlabel("X")
    ax.set_ylabel("Y")
    plt.show()
    
def bev_img(bev_map):
    colored_data = LABEL_COLORS[bev_map.astype(int)]
    print(colored_data.shape)
    img = im.fromarray(colored_data, 'RGB')
    return img

In [12]:
save_dir = os.path.join(file_path, "BEV")
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
for i in range(0, 5000):
    probs_file = os.path.join(file_path, str(i).zfill(6) + ".bin")

    bev_preds = np.fromfile(probs_file, dtype=np.float32).reshape(256, 256, 11)

    bev_map = np.argmax(bev_preds, axis=2)

    img = bev_img(bev_map)
    save_path = os.path.join(save_dir, str(i) + ".png")
    break
    img.save(save_path)
    

(256, 256, 3)
